In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch_geometric
!pip install pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=67a56f4cf11b5bf62875e018570e79c3824f4fa33bba4dde4282d6b9279916d3
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx


def check_if_adj(colors,p1,p2,color_adj):
    h,w = colors.shape
    x1,y1 =p1
    x2, y2 = p2
    x1 = int(x1)
    x2 = int(x2)
    y1 = int(y1)
    y2 = int(y2)
    c1 = colors[h-y1,x1]
    c2 = colors[h-y2,x2]
    if c1 == 0 or c2 == 0:
        return False

    if c2 in color_adj[int(c1)]:
        return True

def check_if_equal(p1,p2,font):
    x1,y1 = p1
    x2,y2 = p2
    if ((x1-x2)**2) + ((y1-y2)**2) <= font**2:
        return True
    return False

def find_font(h,w,img):
    S = np.zeros((h,w))
    for r in range(h):
        if img[r,0] == 0:
            S[r,0] = 1
        else:
            S[r,0] = 0
    for c in range(w):
        if img[0,c] == 0:
            S[0,c] = 1
        else:
            S[0,c] = 0



    for i in range(1,h):
        for j in range(1,w):
            if img[i,j] == 0:
                S[i,j] = min(S[i,j-1],S[i-1,j],S[i-1,j-1])+1
            else:
                S[i,j] = 0
    return np.max(S)




def generate_points(image_path):
    image = Image.open(image_path)


    gray_img = image.convert("L")


    X_s = []
    Y_s = []

    gray_img = np.asarray(gray_img)
    h, w = gray_img.shape
    for y in range(h):
        for x in range(w):
            if gray_img[y,x] < 128:
                X_s.append(x)
                Y_s.append(h - y)



    img = np.asarray(gray_img)
    colors = np.zeros((h,w))
    X = np.asarray(X_s)
    Y = np.asarray(Y_s)

    All_X = X
    All_Y = Y

    font_size = 5* find_font(h,w,img)

    ind_x = []
    ind_y = []
    for i in range(int(h/font_size)+1):
        for j in range(int(w/font_size)+1):
            ind_x.append(j*font_size)
            ind_y.append(i*font_size)

    categ_x = [[] for i in range(len(ind_x))]
    categ_y = [[] for i in range(len(ind_y))]
    ind_x = np.asarray(ind_x)
    ind_y = np.asarray(ind_y)
    for i in range(len(X)):
        x1 = X[i]
        y1 = Y[i]
        temp_x = ind_x - x1
        temp_y = ind_y - y1
        dist = np.square(temp_x) + np.square(temp_y)
        min_ind = np.argmin(dist)
        categ_x[min_ind].append(x1)
        categ_y[min_ind].append(y1)

    categ_x = [i for i in categ_x if i != [] or len(i) > 10]
    categ_y = [i for i in categ_y if i != []or len(i) > 10]

    fin_x = []
    fin_y = []

    for i in range(len(categ_x)):
        mean_x = sum(categ_x[i])/len(categ_x[i])
        mean_y = sum(categ_y[i])/len(categ_y[i])

        if img[h-int(mean_y),int(mean_x)] != 0:
            p = []
            for j in range(len(categ_x[i])):
                p.append((categ_x[i][j],categ_y[i][j]))
            p.sort()
            fin_x.append(p[int(len(categ_x[i])/2)][0])
            fin_y.append(p[int(len(categ_x[i])/2)][1])
        else:
            fin_x.append(mean_x)
            fin_y.append(mean_y)


    for i in range(len(categ_x)):
        for j in range(len(categ_x[i])):
            colors[h-categ_y[i][j],categ_x[i][j]] = i+1

    return fin_x,fin_y,All_X,All_Y,find_font(h,w,img),colors





def around_ones(mat,color_adj):
    center = mat[1,1]
    l = [mat[0,0],mat[0,1],mat[0,2],mat[1,2],mat[2,2],mat[2,1],mat[2,0],mat[1,0]]
    for i in l:
        if l != 0:
            color_adj[int(center)].add(i)


def dfs(start, adj_list,par,parents,visited,path):
    if path[-1] == [] and par != start:
        path[-1].append(par)
    path[-1].append(start)
    visited[start] = 1
    parents[start] = par
    if start not in adj_list.keys():
        return
    for adj in adj_list[start]:
        if visited[adj] == 0:
            dfs(adj,adj_list,start,parents,visited,path)
            if path[-1] != []:
                path.append([])




def graph_making_v3(img_name):
    fin_x, fin_y,_,_,_,colors = generate_points(img_name)

    color_adj = [set() for i in range(len(fin_x)+1)]



    #collor differnt points

    h,w = colors.shape
    for i in range(1,h-1):
        for j in range(1,w-1):
            if colors[i,j] != 0:
                around_ones(colors[i-1:i+2,j-1:j+2],color_adj)


    landmarks = []

    for i in range(len(fin_x)):
        landmarks.append((fin_x[i],fin_y[i]))

    pair_names = [0]*len(fin_x)
    point_named = [-1]*(len(fin_x)+1)

    name_count = 1
    # plt.imshow(colors)
    # plt.show()
    #landmarks.sort(reverse=True)
    adj_list = dict()
    for i in range(len(landmarks)):
        if pair_names[i] == 0:
            pair_names[i] = name_count
            adj_list[name_count] = set()
            name_count += 1

        for j in range(i+1, len(landmarks)):
            if check_if_adj(colors,landmarks[i],landmarks[j],color_adj):
                if pair_names[j] == 0:
                    pair_names[j] = name_count
                    name_count += 1
                if pair_names[j] not in adj_list.keys():
                    adj_list[pair_names[j]] = set()
                adj_list[pair_names[i]].add(pair_names[j])
                adj_list[pair_names[j]].add(pair_names[i])
    min_x = 10**9
    max_x =0
    min_y =10**9
    max_y = 0

    for i in range(len(fin_x)):
        point_named[pair_names[i]] = landmarks[i]
        x,y = landmarks[i]
        if x > max_x:
          max_x = x
        if y > max_y:
          max_y = y
        if x < min_x:
          min_x = x
        if y < min_y:
          min_y = y


    graph = nx.Graph()
    for key in adj_list.keys():
      graph.add_nodes_from([key])
      graph.nodes[key]['x'] = [(point_named[key][0]-min_x)/(max_x-min_x),(point_named[key][1]-min_y)/(max_y-min_y)]
    for key in adj_list.keys():
      for nei in adj_list[key]:
        graph.add_edge(key,nei)

    return graph



In [ ]:
def find_triangle_midpoint(p1, p2, p3):
    x = (p1[0] + p2[0] + p3[0]) / 3
    y = (p1[1] + p2[1] + p3[1]) / 3
    midpoint = (x, y)
    return midpoint



import math
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.tri as tri
import math

from PIL import Image

import networkx as nx


def find_dir(p1,p2,sep_num):
    x1,y1 = p1
    x2,y2 = p2
    if abs(x1-x2) <= 0.001:
        deg = 90
    else:
        tan = (y2 - y1)/(x2-x1)
        if tan < 0:
            deg = math.degrees(math.atan(tan)) + 180
        elif tan > 0:
            deg = math.degrees(math.atan(tan))
        else:
            deg = 0
    for seg in range(sep_num):
        if deg >= seg*(180/sep_num) and deg < (seg+1)*(180/sep_num):
            return seg + 1



def calculate_degree(center, point):
    angle_rad = math.atan2(point[1] - center[1], point[0] - center[0])
    angle_deg = math.degrees(angle_rad)
    angle_deg = (angle_deg + 360) % 360
    return angle_deg


def circle_center(p1, p2, p3):
    """
    Returns the center and radius of the circle passing the given 3 points.
    In case the 3 points form a line, returns (None, infinity).
    """
    temp = p2[0] * p2[0] + p2[1] * p2[1]
    bc = (p1[0] * p1[0] + p1[1] * p1[1] - temp) / 2
    cd = (temp - p3[0] * p3[0] - p3[1] * p3[1]) / 2
    det = (p1[0] - p2[0]) * (p2[1] - p3[1]) - (p2[0] - p3[0]) * (p1[1] - p2[1])

    if abs(det) < 1.0e-6:
        return None

    # Center of circle
    cx = (bc*(p2[1] - p3[1]) - cd*(p1[1] - p2[1])) / det
    cy = ((p1[0] - p2[0]) * cd - (p2[0] - p3[0]) * bc) / det

    return (cx, cy)

def graph_making_v2(img_name):
    image = Image.open(img_name)

    gray_img = image.convert("L")


    X_s = []
    Y_s = []

    gray_img = np.asarray(gray_img)
    h, w = gray_img.shape
    for y in range(1,h,1):
        for x in range(1,w,1):
            if (gray_img[y,x] < 128 and gray_img[y-1,x] > 128) or (gray_img[y,x] > 128 and gray_img[y-1,x] <128) or (gray_img[y,x-1] > 128 and gray_img[y,x] <128) or (gray_img[y,x-1] < 128 and gray_img[y,x] > 128):
                X_s.append(x)
                Y_s.append(h - y)

    X_s = np.asarray(X_s)
    Y_s = np.asarray(Y_s)


    # Remove the super triangle on the outside

    # Helps in determining the neighbours of triangles. I felt it might help in the future
    # Remove this to speed up triangle generation
    # DT.Find_Neighbours()


    # Creating a Triangulation without specifying the triangles results in the
    # Delaunay triangulation of the points.

    # Create the Triangulation; no triangles so Delaunay triangulation created.

    triang = tri.Triangulation(X_s, Y_s)


    X = triang.x
    Y = triang.y
    tries = triang.triangles
    remains = []
    centers_x = []
    centers_y = []
    removed_tries_index = set()
    for i,tr in enumerate(tries):
        x1,x2,x3 = X[tr]
        y1,y2,y3 = Y[tr]
        x, y = find_triangle_midpoint((x1,y1),(x2,y2),(x3,y3))
        if y >= h or x >= w or x < 0 or y < 0:
            removed_tries_index.add(i)
            continue
        if gray_img[int(h-y),int(x)] == 0:
            remains.append(tr)
            centers_x.append(x)
            centers_y.append(y)
        else:
            removed_tries_index.add(i)

    for i,tr in enumerate(tries):
        neighs = list(triang.neighbors[i])
        c = 0
        for nei in neighs:
            if nei == -1 or nei in removed_tries_index:
                c +=1
        if c >= 2:
            removed_tries_index.add(i)

    new_remains = []
    for i,tr in enumerate(tries):
        new_neighs = []
        if i not in removed_tries_index:
            neighs = list(triang.neighbors[i])
            c = 0
            for nei in neighs:
                if nei == -1 or nei in removed_tries_index:
                    c +=1
                    new_neighs.append(-1)
                else:
                    new_neighs.append(nei)
            if c <=2:
                new_remains.append((tr,new_neighs,i))

    index_node_names = [0]*len(tries)



    graph = nx.Graph()
    name_counter = 1
    for tup in new_remains:
        tr, neighs, index = tup
        if index_node_names[index] == 0:
            index_node_names[index] = name_counter
            name_counter += 1
        n_attr = []
        x1,x2,x3 = X[tries[index]]
        y1,y2,y3 = Y[tries[index]]
        p1 = find_triangle_midpoint((x1,y1),(x2,y2),(x3,y3))
        for nei in neighs:
            if nei != -1:
                if index_node_names[nei] == 0:
                    index_node_names[nei] = name_counter
                    name_counter += 1

                x11,x22,x33 = X[tries[nei]]
                y11,y22,y33 = Y[tries[nei]]
                p2 = find_triangle_midpoint((x11,y11),(x22,y22),(x33,y33))
                graph.add_edge(index_node_names[index],index_node_names[nei])
                n_attr.append(calculate_degree(p1,p2)/360.0)
            else:
                n_attr.append(-1)
        n_attr.sort()
        graph.nodes[index_node_names[index]]['x'] = n_attr


    # labels = nx.get_node_attributes(graph, 'value')
    # nx.draw(graph ,labels=labels,with_labels=True, node_color='lightblue', node_size=100, font_size=12, font_color='black', edge_color='gray')
    # plt.show()
    return graph

In [ ]:
def draw_graph(g):
  X_i = []
  Y_i = []
  for node in g.nodes():
    X_i.append(g.nodes[node]['x'][0])
    Y_i.append(g.nodes[node]['x'][1])
  X_s = [[],[]]
  Y_s = [[],[]]
  dic = g.adjacency()
  for key in dic:
    for nei in key[1]:
      X_s[0].append(g.nodes[key[0]]['x'][0])
      X_s[1].append(g.nodes[nei]['x'][0])
      Y_s[0].append(g.nodes[key[0]]['x'][1])
      Y_s[1].append(g.nodes[nei]['x'][1])
  plt.plot(X_s,Y_s)
  plt.scatter(X_i,Y_i)
  plt.xlim(0,1)
  plt.ylim(0,1)
  plt.show()
  return


def find_triangle_midpoint(p1, p2, p3):
    x = (p1[0] + p2[0] + p3[0]) / 3
    y = (p1[1] + p2[1] + p3[1]) / 3
    midpoint = (x, y)
    return midpoint



import math
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.tri as tri
import math

from PIL import Image

import networkx as nx


def find_dir(p1,p2,sep_num):
    x1,y1 = p1
    x2,y2 = p2
    if abs(x1-x2) <= 0.001:
        deg = 90
    else:
        tan = (y2 - y1)/(x2-x1)
        if tan < 0:
            deg = math.degrees(math.atan(tan)) + 180
        elif tan > 0:
            deg = math.degrees(math.atan(tan))
        else:
            deg = 0
    for seg in range(sep_num):
        if deg >= seg*(180/sep_num) and deg < (seg+1)*(180/sep_num):
            return seg + 1



def calculate_degree(center, point):
    angle_rad = math.atan2(point[1] - center[1], point[0] - center[0])
    angle_deg = math.degrees(angle_rad)
    angle_deg = (angle_deg + 360) % 360
    return angle_deg


def circle_center(p1, p2, p3):
    """
    Returns the center and radius of the circle passing the given 3 points.
    In case the 3 points form a line, returns (None, infinity).
    """
    temp = p2[0] * p2[0] + p2[1] * p2[1]
    bc = (p1[0] * p1[0] + p1[1] * p1[1] - temp) / 2
    cd = (temp - p3[0] * p3[0] - p3[1] * p3[1]) / 2
    det = (p1[0] - p2[0]) * (p2[1] - p3[1]) - (p2[0] - p3[0]) * (p1[1] - p2[1])

    if abs(det) < 1.0e-6:
        return None

    # Center of circle
    cx = (bc*(p2[1] - p3[1]) - cd*(p1[1] - p2[1])) / det
    cy = ((p1[0] - p2[0]) * cd - (p2[0] - p3[0]) * bc) / det

    return (cx, cy)

def graph_making(img_name):
    image = Image.open(img_name)

    gray_img = image.convert("L")


    X_s = []
    Y_s = []

    gray_img = np.asarray(gray_img)
    h, w = gray_img.shape
    for y in range(1,h,1):
        for x in range(1,w,1):
            if (gray_img[y,x] < 128 and gray_img[y-1,x] > 128) or (gray_img[y,x] > 128 and gray_img[y-1,x] <128) or (gray_img[y,x-1] > 128 and gray_img[y,x] <128) or (gray_img[y,x-1] < 128 and gray_img[y,x] > 128):
                X_s.append(x)
                Y_s.append(h - y)

    X_s = np.asarray(X_s)
    Y_s = np.asarray(Y_s)


    # Remove the super triangle on the outside

    # Helps in determining the neighbours of triangles. I felt it might help in the future
    # Remove this to speed up triangle generation
    # DT.Find_Neighbours()


    # Creating a Triangulation without specifying the triangles results in the
    # Delaunay triangulation of the points.

    # Create the Triangulation; no triangles so Delaunay triangulation created.

    triang = tri.Triangulation(X_s, Y_s)


    X = triang.x
    Y = triang.y
    tries = triang.triangles
    remains = []
    centers_x = []
    centers_y = []
    removed_tries_index = set()
    for i,tr in enumerate(tries):
        x1,x2,x3 = X[tr]
        y1,y2,y3 = Y[tr]
        x, y = find_triangle_midpoint((x1,y1),(x2,y2),(x3,y3))
        if y >= h or x >= w or x < 0 or y < 0:
            removed_tries_index.add(i)
            continue
        if gray_img[int(h-y),int(x)] == 0:
            remains.append(tr)

            centers_x.append(x)
            centers_y.append(y)
        else:
            removed_tries_index.add(i)

    for i,tr in enumerate(tries):
        neighs = list(triang.neighbors[i])
        c = 0
        for nei in neighs:
            if nei == -1 or nei in removed_tries_index:
                c +=1
        if c >= 2:
            removed_tries_index.add(i)

    new_remains = []
    for i,tr in enumerate(tries):
        new_neighs = []
        if i not in removed_tries_index:
            neighs = list(triang.neighbors[i])
            c = 0
            for nei in neighs:
                if nei == -1 or nei in removed_tries_index:
                    c +=1
                    new_neighs.append(-1)
                else:
                    new_neighs.append(nei)
            if c <=2:
                new_remains.append((tr,new_neighs,i))

    index_node_names = [0]*len(tries)



    graph = nx.Graph()
    name_counter = 1
    max_x = 0
    max_y = 0
    for tup in new_remains:
        tr, neighs, index = tup
        if index_node_names[index] == 0:
            index_node_names[index] = name_counter
            name_counter += 1
        x1,x2,x3 = X[tries[index]]
        y1,y2,y3 = Y[tries[index]]
        p1 = find_triangle_midpoint((x1,y1),(x2,y2),(x3,y3))
        if p1[0] > max_x:
          max_x = p1[0]
        if p1[1] > max_y:
          max_y = p1[1]

        for nei in neighs:
            if nei != -1:
                if index_node_names[nei] == 0:
                    index_node_names[nei] = name_counter
                    name_counter += 1

                x11,x22,x33 = X[tries[nei]]
                y11,y22,y33 = Y[tries[nei]]
                p2 = find_triangle_midpoint((x11,y11),(x22,y22),(x33,y33))
                graph.add_edge(index_node_names[index],index_node_names[nei])

        graph.nodes[index_node_names[index]]['x'] = [p1[0],p1[1]]

    graph.add_node(10000)
    graph.nodes[10000]['x'] = [0,0]
    labels = []
    for node in graph.nodes:
        graph.nodes[node]['x'] = [graph.nodes[node]['x'][0]/max_x,graph.nodes[node]['x'][1]/max_y]
        labels.append((graph.nodes[node]['x'][0]/max_x,graph.nodes[node]['x'][1]/max_y))
        graph.add_edge(10000,node)







    nx.draw(graph ,labels=labels,with_labels=True, node_color='lightblue', node_size=100, font_size=12, font_color='black', edge_color='gray')
    plt.show()
    return graph



graph = graph_making("/content/drive/MyDrive/ImageDS/alef_v1.png")




FileNotFoundError: ignored

In [ ]:
from torch_geometric.utils import from_networkx
import torch
def one_hot_encode(n_c,number):
  l = [0]*n_c
  l[number-1] = 1
  return l

graph_data_list = []
graph_names = ['alef','ain','sad','sin','re','de','me','le','be']

for i,g_n in enumerate(graph_names):
  for j in range(10):
    graph = graph_making_v3("/content/drive/MyDrive/ImageDS/"+g_n+"_v"+str(j+1)+".png")
    data = from_networkx(graph)
    data.y = torch.tensor(one_hot_encode(len(graph_names),i+1))
    graph_data_list.append(data)
for d in graph_data_list:
  print(d)

In [ ]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(graph_data_list, batch_size=16, shuffle=True)
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()



In [ ]:
from torch_geometric.nn import GraphConv,Linear
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import global_mean_pool

class GNN(torch.nn.Module):
    def __init__(self,feature_numbers ,hidden_channels,num_classes):
        super(GNN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphConv(feature_numbers,hidden_channels)
        self.conv2 = GraphConv(hidden_channels,hidden_channels)
        self.conv3 = GraphConv(hidden_channels,hidden_channels)
        self.lin = Linear(hidden_channels,num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        x = global_mean_pool(x, batch)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return x



In [ ]:
model = GNN(feature_numbers= 2,hidden_channels=64,num_classes = 9)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:
         out = model(data.x.float(), data.edge_index, data.batch)
         y = data.y.view(-1,9)
         loss = criterion(out.float(), y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x.float(), data.edge_index, data.batch)
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         y = data.y.view(-1,9).argmax(dim=1)

         correct += int((pred == y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    train()
    train_acc = test(train_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')


Epoch: 022, Train Acc: 0.4111
Epoch: 023, Train Acc: 0.4222
Epoch: 024, Train Acc: 0.4667
Epoch: 025, Train Acc: 0.4444
Epoch: 026, Train Acc: 0.3667
Epoch: 027, Train Acc: 0.4667
Epoch: 028, Train Acc: 0.3667
Epoch: 029, Train Acc: 0.4889
Epoch: 030, Train Acc: 0.5778
Epoch: 031, Train Acc: 0.5556
Epoch: 032, Train Acc: 0.5889
Epoch: 033, Train Acc: 0.5556
Epoch: 034, Train Acc: 0.5889
Epoch: 035, Train Acc: 0.5667
Epoch: 036, Train Acc: 0.6000
Epoch: 037, Train Acc: 0.5667
Epoch: 038, Train Acc: 0.6333
Epoch: 039, Train Acc: 0.6222
Epoch: 040, Train Acc: 0.5333
Epoch: 041, Train Acc: 0.6667
Epoch: 042, Train Acc: 0.6333
Epoch: 043, Train Acc: 0.6667
Epoch: 044, Train Acc: 0.7111
Epoch: 045, Train Acc: 0.6667
Epoch: 046, Train Acc: 0.6111
Epoch: 047, Train Acc: 0.6667
Epoch: 048, Train Acc: 0.6111
Epoch: 049, Train Acc: 0.4889
Epoch: 050, Train Acc: 0.6444
Epoch: 051, Train Acc: 0.6000
Epoch: 052, Train Acc: 0.6778
Epoch: 053, Train Acc: 0.5444
Epoch: 054, Train Acc: 0.6111
Epoch: 055

In [ ]:
graph = graph_making_v3("/content/drive/MyDrive/testImgDS/t2.png")
data = from_networkx(graph)
model.eval()
output = model(data.x.float(),data.edge_index,data.batch)
l = output.argmax(dim=1)
print(graph_names[l])
